In [ ]:
# 最低限必要なデータ
# 物件名　住所　間取り　家賃　階数

### スクレイピング

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm   #for文の進捗を確認

# スクレイピングを行うURLの指定
REQUEST_URL = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&ta=13&bs=040&ekInput=28860&tj=30&nk=-1&ekInput=17460&tj=30&nk=-1&ct=25.0&cb=0.0&et=9999999&mt=9999999&mb=0&cn=15&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&pc=30"

# ページ数の取得
res = requests.get(REQUEST_URL)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")
maxpage = int(soup.select("ol.pagination-parts a")[-1].text) #"-1"でリストの一番最後の文字列を取り出す→整数化

#ステータスコードが正常であれば実行
if res.status_code == 200:

    # 各データを格納するためのリスト
    names = []
    addresses = []
    station_info1_list = []
    station_info2_list = []
    station_info3_list = []
    years = []
    maxfloors = []

    floors = []
    rent_fees = []
    maintenance_fees = []
    deposit_fees = []
    gratuity_fees = []
    layouts = []
    areas = []

    # ページごとのスクレイピング
    for k in tqdm(range(int(maxpage))): #for文の進捗を確認
        res = requests.get(REQUEST_URL+"&page="+str(k)) # ページを変更したURLにアクセス
        res.encoding = "utf-8"
        soup = BeautifulSoup(res.text, "html.parser")
        details = soup.select("div.cassetteitem-detail")
        time.sleep(1)   #サーバー負荷を避けるため1s遅延

        # 各物件のデータを取得（上部）
        for i in range(len(details)):
            detail = details[i]
            name = detail.select("div.cassetteitem_content-title")[0].text
            address = detail.select("li.cassetteitem_detail-col1")[0].text
            station_info1 = detail.select("div.cassetteitem_detail-text")[0].text
            station_info2 = detail.select("div.cassetteitem_detail-text")[1].text
            station_info3 = detail.select("div.cassetteitem_detail-text")[2].text
            year, maxfloor = detail.select("li.cassetteitem_detail-col3")[0].text.split()
            
            # print("--------------------------------------------------")
            # print(name)
            # print(address)
            # print(station_info1)
            # print(station_info2)
            # print(station_info3)
            # print(year)
            # print(maxfloor)

            # 物件の詳細情報（下部）
            items = soup.select("div.cassetteitem-item")[i].select("tbody")
            for j in range(len(items)):
                item = items[j]

                # 各項目のデータを取得（下部）
                floor = item.select("td")[2].text.split()[0]
                rent_fee, maintenance_fee = item.select("td")[3].text.split()
                deposit_fee, gratuity_fee = item.select("td")[4].text.split()
                layout, area = item.select("td")[5].text.split()

                # print("---------")
                # print(floor)
                # print(rent_fee)
                # print(maintenance_fee)
                # print(deposit_fee)
                # print(gratuity_fee)
                # print(layout)
                # print(area)

                # 取得したデータをリストに追加
                names.append(name)
                addresses.append(address)
                station_info1_list.append(station_info1)
                station_info2_list.append(station_info2)
                station_info3_list.append(station_info3)
                years.append(year)
                maxfloors.append(maxfloor)

                floors.append(floor)
                rent_fees.append(rent_fee)
                maintenance_fees.append(maintenance_fee)
                deposit_fees.append(deposit_fee)
                gratuity_fees.append(gratuity_fee)
                layouts.append(layout)
                areas.append(area)

    # 取得したデータをdfに格納
    data = {
        'Name': names,
        'Address': addresses,
        'Station_Info1': station_info1_list,
        'Station_Info2': station_info2_list,
        'Station_Info3': station_info3_list,
        'Year': years,
        'Max_Floor': maxfloors,
        'Floor': floors,
        'Rent_Fee': rent_fees,
        'Maintenance_Fee': maintenance_fees,
        'Deposit_Fee': deposit_fees,
        'Gratuity_Fee': gratuity_fees,
        'Layout': layouts,
        'Area': areas
    }
    df = pd.DataFrame(data)
    # print(df)

100%|██████████| 515/515 [31:43<00:00,  3.70s/it]


### スプレッドシートへの転記

In [28]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

# GoogleSheetsAPI、GoogleDriveAPI、及び認証鍵の指定
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

#↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓jsonファイル名指定↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
SERVICE_ACCOUNT_FILE=""

# Service Accountの認証情報を取得
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE,SCOPES)

# 認証情報を用いてGoogleSheetsにアクセス
gs = gspread.authorize(credentials)

# 対象のスプレッドシートとワークシートを指定
#↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓シートキー指定↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
SPERADSHEET_KEY=""
workbook=gs.open_by_key(SPERADSHEET_KEY)
worksheet=workbook.worksheet("元データ")

#dfから値を習得
values = [df.columns.values.tolist()] + df.values.tolist()

# ワークシートの指定したセル(B2)から値を追加
worksheet.update("B2", values)

C:\Users\zk-ht\AppData\Local\Temp\ipykernel_2612\926802016.py:24: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update("B2", values)


{'spreadsheetId': '1E5sSVGEaVJvmI-gU48-6glk5WtS7wq0nPL6i5FDPd5A',
 'updatedRange': "'元データ'!B2:O38300",
 'updatedRows': 38299,
 'updatedColumns': 14,
 'updatedCells': 536186}